# Preprocess the training and national validation data

In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()
data_path = os.getenv('RAW_DATA')

df = pd.read_csv(data_path)
df = df.loc[df['Imputation_'] == 0]
df.columns

Index(['Imputation_', 'ID_progressivo', 'ID', 'Studio', 'Visite', 'Duplicati',
       'Data_v1', 'Centro', 'Iniz', 'Data_nascita',
       ...
       'ESRD', 'Death_preHD', 'fu_esrd_2', 'fu_morte_2', 'fu_esrd_1',
       'fu_morte_1', 'time12', 'stadi', 'filter_$', 'stadi_345'],
      dtype='object', length=102)

In [2]:
print(df.shape)
print((df['Imputation_'] == 0).sum())

(3957, 102)
3957


### Preprocess the data

In [3]:
# Select only useful columns

df = df[['ID_progressivo', 'Visite', 'Studio', 'Centro', 'Sesso', 'Altezza', 'Peso', 'Diabete',
       'Diagnosi_new', 'IMA', 'Ictus', 'Angina', 'PVD', 'CHF', 'CVD', 'Fumo',
       'ESRD', 'stadi', 'Eta1', 'Pas1', 'Pad1', 'Prot1', 'Sodiuria1',
       'Creat1', 'Creatininuria1', 'Epi1', 'prot_gfr1', 'Trigliceridi1',
       'Colesterolo1', 'Ldl1', 'Hdl1', 'Glicemia1', 'Hb1', 'Tsat1',
       'Ferritina1', 'Ca1', 'P1', 'PTH1', 'Albumina1', 'BMI1', 'CCB1', 'ACE1',
       'ARB1', 'RAS1', 'BB1', 'All_diuretici1', 'N_farmaci1', 'ESA1', 'VitD1',
       'Statine1', 'fu_esrd_1']]

# One hot encode CKD cause
df['ckd_cause_hypertens'] = np.where(df['Diagnosi_new'] == 1, 1, 0)
df['ckd_cause_diabet'] = np.where(df['Diagnosi_new'] == 2, 1, 0)
df['ckd_cause_glom_dis'] = np.where(df['Diagnosi_new'] == 3, 1, 0)
df['ckd_cause_tubul_inter'] = np.where(df['Diagnosi_new'] == 4, 1, 0)
df['ckd_cause_pkd'] = np.where(df['Diagnosi_new'] == 5, 1, 0)

# Prepare follow-up data for training
df['esrd_upper'] = np.where(df['ESRD'] == 1, df['fu_esrd_1'], np.where(df['ESRD'] == 0, np.inf, df['fu_esrd_1']))
df['esrd_lower'] = df['fu_esrd_1']

# Drop unncessary columns
df = df.drop(columns = ['ID_progressivo', 'Visite', 'Centro',
                        'fu_esrd_1', 'VitD1', 'Hdl1', 'Ferritina1', 'Tsat1',
                        'PTH1', 'prot_gfr1', 'stadi', 'ESA1', 'N_farmaci1', 
                        'Creatininuria1', 'ACE1', 'ARB1', 'Statine1', 'BB1',
                        'CCB1', 'Diagnosi_new'])

df.columns

Index(['Studio', 'Sesso', 'Altezza', 'Peso', 'Diabete', 'IMA', 'Ictus',
       'Angina', 'PVD', 'CHF', 'CVD', 'Fumo', 'ESRD', 'Eta1', 'Pas1', 'Pad1',
       'Prot1', 'Sodiuria1', 'Creat1', 'Epi1', 'Trigliceridi1', 'Colesterolo1',
       'Ldl1', 'Glicemia1', 'Hb1', 'Ca1', 'P1', 'Albumina1', 'BMI1', 'RAS1',
       'All_diuretici1', 'ckd_cause_hypertens', 'ckd_cause_diabet',
       'ckd_cause_glom_dis', 'ckd_cause_tubul_inter', 'ckd_cause_pkd',
       'esrd_upper', 'esrd_lower'],
      dtype='object')

In [4]:
print('Percent of missings per feature:\n')

for col in df.columns:
    print(col + ': ' + str(round((sum(np.isnan(df[col])) / len(df)) * 100, 1)) + '%')

Percent of missings per feature:

Studio: 0.0%
Sesso: 0.0%
Altezza: 0.3%
Peso: 0.6%
Diabete: 0.0%
IMA: 11.4%
Ictus: 11.4%
Angina: 16.0%
PVD: 11.4%
CHF: 11.4%
CVD: 9.3%
Fumo: 14.2%
ESRD: 0.0%
Eta1: 0.0%
Pas1: 1.2%
Pad1: 1.2%
Prot1: 0.0%
Sodiuria1: 40.6%
Creat1: 0.0%
Epi1: 0.0%
Trigliceridi1: 11.1%
Colesterolo1: 5.2%
Ldl1: 31.5%
Glicemia1: 10.2%
Hb1: 1.1%
Ca1: 9.0%
P1: 9.1%
Albumina1: 18.1%
BMI1: 1.0%
RAS1: 1.8%
All_diuretici1: 11.8%
ckd_cause_hypertens: 0.0%
ckd_cause_diabet: 0.0%
ckd_cause_glom_dis: 0.0%
ckd_cause_tubul_inter: 0.0%
ckd_cause_pkd: 0.0%
esrd_upper: 0.0%
esrd_lower: 0.0%


In [5]:
# Drop columns with high percent of missings
df = df.drop(columns = ['Sodiuria1', 'Ldl1', 'Trigliceridi1', 'Fumo', 'IMA', 'Ictus', 'Angina', 'PVD', 
                        'CHF', 'CVD', 'Pad1', 'Pas1', 'All_diuretici1', 'Altezza', 'Peso', 'Glicemia1', 
                        'Albumina1', 'RAS1'])
df.columns

Index(['Studio', 'Sesso', 'Diabete', 'ESRD', 'Eta1', 'Prot1', 'Creat1', 'Epi1',
       'Colesterolo1', 'Hb1', 'Ca1', 'P1', 'BMI1', 'ckd_cause_hypertens',
       'ckd_cause_diabet', 'ckd_cause_glom_dis', 'ckd_cause_tubul_inter',
       'ckd_cause_pkd', 'esrd_upper', 'esrd_lower'],
      dtype='object')

In [6]:
print('Percent of missings per feature:\n')

for col in df.columns:
    print(col + ': ' + str(round((sum(np.isnan(df[col])) / len(df)) * 100, 1)) + '%')

Percent of missings per feature:

Studio: 0.0%
Sesso: 0.0%
Diabete: 0.0%
ESRD: 0.0%
Eta1: 0.0%
Prot1: 0.0%
Creat1: 0.0%
Epi1: 0.0%
Colesterolo1: 5.2%
Hb1: 1.1%
Ca1: 9.0%
P1: 9.1%
BMI1: 1.0%
ckd_cause_hypertens: 0.0%
ckd_cause_diabet: 0.0%
ckd_cause_glom_dis: 0.0%
ckd_cause_tubul_inter: 0.0%
ckd_cause_pkd: 0.0%
esrd_upper: 0.0%
esrd_lower: 0.0%


In [7]:
num_vars = []
cat_vars = []
print('Unique values per feature\n')
for col in df.columns:
    if (len(np.unique(df[col])) >= 10) & (sum(np.isnan(df[col])) != 0):
        num_vars.append(col)
    elif (len(np.unique(df[col])) < 10) & (sum(np.isnan(df[col])) != 0):
        cat_vars.append(col)
    print(col + ': ' + str(len(np.unique(df[col]))))

print('\n------------\nNumerical variables:')
print(num_vars)
print('\n------------\nCategorical variables:')
print(cat_vars)

Unique values per feature

Studio: 6
Sesso: 2
Diabete: 2
ESRD: 2
Eta1: 3569
Prot1: 892
Creat1: 383
Epi1: 3952
Colesterolo1: 299
Hb1: 110
Ca1: 59
P1: 70
BMI1: 2088
ckd_cause_hypertens: 2
ckd_cause_diabet: 2
ckd_cause_glom_dis: 2
ckd_cause_tubul_inter: 2
ckd_cause_pkd: 2
esrd_upper: 704
esrd_lower: 1956

------------
Numerical variables:
['Colesterolo1', 'Hb1', 'Ca1', 'P1', 'BMI1']

------------
Categorical variables:
[]


In [8]:
# Use cohort 3 as a test set

train = df.loc[df['Studio'] != 7]
test = df.loc[df['Studio'] == 7]

In [9]:
from statistics import mode
# Impute numerical variables

for num_var in num_vars:
    train_index = train.index
    train.loc[train_index, num_var] = train[num_var].fillna(train[num_var].median())

for cat_var in cat_vars:
    train_index = train.index
    train.loc[train_index, cat_var] = train[cat_var].fillna(mode(train[cat_var]))

In [10]:
print('Percent of missings per feature for the training set:\n')
for col in train.columns:
    print(col + ': ' + str(round((sum(np.isnan(train[col])) / len(train)) * 100, 1)) + '%')

Percent of missings per feature for the training set:

Studio: 0.0%
Sesso: 0.0%
Diabete: 0.0%
ESRD: 0.0%
Eta1: 0.0%
Prot1: 0.0%
Creat1: 0.0%
Epi1: 0.0%
Colesterolo1: 0.0%
Hb1: 0.0%
Ca1: 0.0%
P1: 0.0%
BMI1: 0.0%
ckd_cause_hypertens: 0.0%
ckd_cause_diabet: 0.0%
ckd_cause_glom_dis: 0.0%
ckd_cause_tubul_inter: 0.0%
ckd_cause_pkd: 0.0%
esrd_upper: 0.0%
esrd_lower: 0.0%


In [11]:
print('Percent of missings per feature for the test set:\n')
for col in train.columns:
    print(col + ': ' + str(round((sum(np.isnan(test[col])) / len(test)) * 100, 1)) + '%')

Percent of missings per feature for the test set:

Studio: 0.0%
Sesso: 0.0%
Diabete: 0.0%
ESRD: 0.0%
Eta1: 0.0%
Prot1: 0.0%
Creat1: 0.0%
Epi1: 0.0%
Colesterolo1: 0.0%
Hb1: 1.1%
Ca1: 0.6%
P1: 1.4%
BMI1: 0.3%
ckd_cause_hypertens: 0.0%
ckd_cause_diabet: 0.0%
ckd_cause_glom_dis: 0.0%
ckd_cause_tubul_inter: 0.0%
ckd_cause_pkd: 0.0%
esrd_upper: 0.0%
esrd_lower: 0.0%


In [12]:
train_data_path = os.getenv('TRAINING_DATA')
nat_val_data_path = os.getenv('NATIONAL_VAL')
train.to_csv(train_data_path, index = False)
test.to_csv(nat_val_data_path, index = False)